In [1]:
import sys
rdd = sc.parallelize(xrange(10),10)
aa = rdd.map(lambda x: sys.version)
print aa.count()

10


In [9]:
import os
print os.getcwd()

/Users/hongyili/Untitled Folder


In [48]:
sys.path.append('/Users/hongyili/Untitled Folder')
import geohash

In [50]:
y_rdd = sc.textFile("/Users/hongyili/Downloads/yellow*.csv")
y_rdd = y_rdd.map(lambda line: tuple(line.split(',')))

g_rdd = sc.textFile("/Users/hongyili/Downloads/green*.csv")
g_rdd = g_rdd.map(lambda line: tuple(line.split(',')))

In [51]:
import time
from datetime import date
import math

def date_extractor(date_str,b,minutes_per_bin):
    # Takes a datetime object as a parameter
    # and extracts and returns a tuple of the form: (as per the data specification)
    # (time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)
    # Split date string into list of date, time
    
    d = date_str.split()
    
    #safety check
    if len(d) != 2:
        return tuple([None,])
    
    # TIME (eg. for 16:56:20 and 15 mins per bin)
    #list of hour,min,sec (e.g. [16,56,20])
    time_list = [int(t) for t in d[1].split(':')]
    
    #safety check
    if len(time_list) != 3:
        return tuple([None,])
    
    # calculate number of minute into the day (eg. 1016)
    num_minutes = time_list[0] * 60 + time_list[1]
    
    # Time of the start of the bin
    time_bin = num_minutes / minutes_per_bin     # eg. 1005
    hour_bin = num_minutes / 60                  # eg. 16
    min_bin = (time_bin * minutes_per_bin) % 60  # eg. 45
    
    #get time_cat
    hour_str = str(hour_bin) if hour_bin / 10 > 0 else "0" + str(hour_bin)  # eg. "16"
    min_str = str(min_bin) if min_bin / 10 > 0 else "0" + str(min_bin)      # eg. "45"
    time_cat = hour_str + ":" + min_str                                     # eg. "16:45"
    
    # Get a floating point representation of the center of the time bin
    time_num = (hour_bin*60 + min_bin + minutes_per_bin / 2.0)/(60*24)      # eg. 0.7065972222222222
    
    time_cos = math.cos(time_num * 2 * math.pi)
    time_sin = math.sin(time_num * 2 * math.pi)
    
    # DATE
    # Parse year, month, day
    date_list = d[0].split('-')
    d_obj = date(int(date_list[0]),int(date_list[1]),int(date_list[2]))
    day_to_str = {0: "Monday",
                  1: "Tuesday",
                  2: "Wednesday",
                  3: "Thursday",
                  4: "Friday",
                  5: "Saturday",
                  6: "Sunday"}
    day_of_week = d_obj.weekday()
    day_cat = day_to_str[day_of_week]
    day_num = (day_of_week + time_num)/7.0
    day_cos = math.cos(day_num * 2 * math.pi)
    day_sin = math.sin(day_num * 2 * math.pi)
    
    year = d_obj.year
    month = d_obj.month
    day = d_obj.day
    
    weekend = 0
    #check if it is the weekend
    if day_of_week in [5,6]:
        weekend = 1
       
    return (year, month, day, time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)

In [52]:

def data_cleaner(zipped_row):
    # takes a tuple (row,g,b,minutes_per_bin) as a parameter and returns a tuple of the form:
    # (time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend,geohash)
    row = zipped_row[0]
    g = zipped_row[1]
    b = zipped_row[2]
    minutes_per_bin = zipped_row[3]
    # The indices of pickup datetime, longitude, and latitude respectively
    indices = (1, 6, 5)
    
    #safety check: make sure row has enough features
    if len(row) < 7:
        return None
    
    #extract day of the week and hour
    date_str = row[indices[0]]
    clean_date = date_extractor(date_str,b,minutes_per_bin)
    #get geo hash

    latitude = float(row[indices[1]])
    longitude = float(row[indices[2]])
    location = None
    #safety check: make sure latitude and longitude are valid
    if latitude < 41.1 and latitude > 40.5 and longitude < -73.6 and longitude > -74.1:
        location = geohash.encode(latitude,longitude, g)
    else:
        return None

    return tuple(list(clean_date)+[location])

In [53]:
g = 7 #geohash length
b = 48 # number of time bins per day
# Note: b must evenly divide 60
minutes_per_bin = int((24 / float(b)) * 60)

In [54]:
gclean_rdd = g_rdd.map(lambda row: (row, g, b, minutes_per_bin)).map(data_cleaner).filter(lambda row: row != None).map(lambda row: (row,1)).reduceByKey(lambda a,b: a + b).map(lambda row: (row,'g'))          


In [55]:
yclean_rdd = y_rdd.map(lambda row: (row, g, b, minutes_per_bin)).map(data_cleaner).filter(lambda row: row != None).map(lambda row: (row,1)).reduceByKey(lambda a,b: a + b).map(lambda row: (row,'y')) 


In [56]:
import numpy as np
combined_rdd = yclean_rdd.union(gclean_rdd)
final_rdd = combined_rdd.map(lambda row: row[0]).reduceByKey(lambda a,b: a + b).map(lambda (a,b): (a,b,np.random.random()))

In [57]:
train_fraction = 0.1
valid_fraction = 0.25

In [62]:
trainset = final_rdd.filter(lambda (a,b,c): ((a[0] == 2015) | (a[1] == 3)) & (c <= train_fraction))
trainset.repartition(1).saveAsTextFile("/Users/hongyili/Desktop/big_Data_final／trainset11")

In [64]:
validset = final_rdd.filter(lambda (a,b,c): (a[0] == 2015) & (a[1] == 4) & (c <= valid_fraction))
validset.repartition(1).saveAsTextFile("/Users/hongyili/Desktop/big_Data_final／trainset6")

In [65]:
testset  = final_rdd.filter(lambda (a,b,c): (a[0] == 2015) & (a[1] == 5)) 
testset.repartition(1).saveAsTextFile("/Users/hongyili/Desktop/big_Data_final/trainset6")

In [68]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
import geohash
from sklearn.metrics import mean_squared_error
import random

In [69]:
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_, gs.scorer_
    print "Best score: ", gs.best_score_
    best = gs.best_estimator_
    return best

In [70]:
names = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat", "day_num", "day_cos", "day_sin", "weekend", "geohash", "pickups"]
dftaxi=pd.read_csv("/Users/hongyili/Desktop/big_Data_final/train_csv/train.csv", header=None, names = names)
print dftaxi.shape

(868979, 11)


In [71]:
itrain, itest = train_test_split(xrange(dftaxi.shape[0]), train_size=0.8)
mask=np.ones(dftaxi.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

In [72]:
print mask

[ True False  True ...,  True  True  True]


In [73]:
Xnames = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat",
          "day_num", "day_cos", "day_sin", "weekend", "geohash"]
X = dftaxi[Xnames]

y = np.log10(dftaxi['pickups']+1)

In [74]:
def decodegeo(geo, which):
    if len(geo) >= 6:
        geodecoded = geohash.decode(geo)
        return geodecoded[which]
    else:
        return 0
X['latitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 0))
X['longitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 1))
X.head()

In [75]:
print X.head()

          time_cat  time_num  time_cos  time_sin    day_cat   day_num   day_cos   day_sin  weekend  geohash   latitude  longitude
2015 4 28    14:30  0.614583 -0.751840 -0.659346    Tuesday  0.230655  0.121251  0.992622        0  dr5ru54  40.754471 -74.000473
     3 19    19:30  0.822917  0.442289 -0.896873   Thursday  0.546131 -0.958287 -0.285808        0  dr5rut6  40.766830 -73.978500
     5 16    00:30  0.031250  0.980785  0.195090   Saturday  0.718750 -0.195090 -0.980785        1  dr5ryj1  40.765457 -73.913956
     3 8     04:00  0.177083  0.442289  0.896873     Sunday  0.882440  0.739379 -0.673289        1  dr5rv38  40.746231 -73.948288
     4 15    03:30  0.156250  0.555570  0.831470  Wednesday  0.308036 -0.356622  0.934249        0  dr5rgcx  40.746231 -74.004593


In [76]:
X = X.drop(['time_cat','day_cat','geohash'], axis=1)

In [77]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]
print Xtrain.shape
max_samples = 1000000
if Xtrain.shape[0] > max_samples:
    rows = random.sample(Xtrain.index, max_samples)
    Xtrain = Xtrain.ix[rows]
    ytrain = ytrain.ix[rows]
print Xtrain.shape
Xtrain.head()

(695183, 9)
(695183, 9)


In [78]:
estimator = RandomForestRegressor(n_estimators=20, n_jobs=-1)

In [79]:
parameters = {"n_estimators": [50],
              "max_features": ["auto"], # ["auto","sqrt","log2"]
              "max_depth": [50]}
best = cv_optimize(estimator, parameters, Xtrain, ytrain, n_folds=5, score_func='mean_squared_error', verbose=3)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.032575 - 2.0min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.032608 - 1.9min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.032664 - 1.8min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.032964 - 1.9min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.032654 - 1.7min
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  9.2min finished
BEST {'max_features': 'auto', 'n_estimators': 50, 'max_depth': 50} -0.0326930578467 [mean: -0.03269, std: 0.000

In [80]:
reg=best.fit(Xtrain, ytrain)
training_accuracy = reg.score(Xtrain, ytrain)
test_accuracy = reg.score(Xtest, ytest)

In [81]:
print test_accuracy

0.713449438461


In [82]:
np.round(np.power(10,np.column_stack((reg.predict(Xtest),ytest))) - 1,decimals=0).astype(int)

In [83]:
rmse = np.sqrt(mean_squared_error(reg.predict(Xtest),ytest))

In [84]:
import operator
dict_feat_imp = dict(zip(list(X.columns.values),reg.feature_importances_))

sorted_features = sorted(dict_feat_imp.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

In [85]:
time_cols = list(X.columns.values)
time_cols.remove('latitude')
time_cols.remove('longitude')
loc_cols = ['latitude', 'longitude']
time_data = X.drop(loc_cols, axis=1).drop_duplicates()
time_data = time_data[time_data['day_num'] <= 1/7.]
loc_data = X.drop(time_cols, axis=1).drop_duplicates()
loc_data = loc_data[(loc_data['latitude'] > 40.5) & (loc_data['latitude'] < 41.1) &
                    (loc_data['longitude'] > -74.1) & (loc_data['longitude'] < -73.6)]

In [86]:
time_data['key'] = 1
loc_data['key'] = 1
result = pd.merge(time_data, loc_data, on='key').drop(['key'], axis=1)
result = result[Xtrain.columns.values]

In [87]:
yy = dftaxi[['geohash','day_num','pickups']]
yy['latitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 0))
yy['longitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 1))
result['pred_pickups'] = np.power(10,reg.predict(result)) - 1
result = pd.merge(result, yy, how='left', on=['day_num', 'latitude', 'longitude']).drop(['geohash'], axis=1)

/var/folders/x3/qp53rvhx52b7gxvpc7dk0gt00000gn/T/kernel-PySpark-d414eddb-389e-437a-a1f7-f7bb7f467b8b/pyspark_runner.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Licensed to the Apache Software Foundation (ASF) under one or more
/var/folders/x3/qp53rvhx52b7gxvpc7dk0gt00000gn/T/kernel-PySpark-d414eddb-389e-437a-a1f7-f7bb7f467b8b/pyspark_runner.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # contributor license agreements.  See the NOTICE file distributed with


In [88]:
print result.head(10)

   time_num  time_cos  time_sin   day_num   day_cos   day_sin  weekend   latitude  longitude  pred_pickups  pickups
0  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  40.754471 -74.000473      1.000000      NaN
1  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  40.766830 -73.978500      4.302656      NaN
2  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  40.765457 -73.913956      1.625663      NaN
3  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  40.746231 -73.948288      1.286634      NaN
4  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  40.746231 -74.004593      1.607528      1.0
5  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  40.825882 -73.935928      1.148326      NaN
6  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  40.794296 -73.971634     11.363229     11.0
7  0.989583  0.997859 -0.065403  0.141369  0.630773  0.775968        0  

In [89]:
result = result.drop(['time_cos','day_num','time_sin','day_cos','day_sin','weekend'], axis=1)

In [90]:
result.to_csv('/Users/hongyili/Desktop/big_Data_final/taxi-data-predictions_prec_7_monday.csv')

In [91]:
yy = dftaxi[['geohash','day_num','pickups']]
yy['latitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 0))
yy['longitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 1))
result['pred_pickups'] = np.power(10,reg.predict(result)) - 1
result = pd.merge(result, yy, how='left', on=['day_num', 'latitude', 'longitude']), axis=1)

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/x3/qp53rvhx52b7gxvpc7dk0gt00000gn/T/kernel-PySpark-d414eddb-389e-437a-a1f7-f7bb7f467b8b/pyspark_runner.py", line 132, in <module>
    compiled_code = compile(final_code, "<string>", "exec")
  File "<string>", line 5
    result = pd.merge(result, yy, how='left', on=['day_num', 'latitude', 'longitude']), axis=1)
                                                                                              ^
SyntaxError: invalid syntax

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:140)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:140)
scala.Option.foreach(Option.scala:236)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:139)
sun.reflect.GeneratedMethodAccessor93.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImpl.invo

In [92]:
yy = dftaxi[['geohash','day_num','pickups']]
yy['latitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 0))
yy['longitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 1))
result['pred_pickups'] = np.power(10,reg.predict(result)) - 1
result = pd.merge(result, yy, how='left', on=['day_num', 'latitude', 'longitude'])

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/x3/qp53rvhx52b7gxvpc7dk0gt00000gn/T/kernel-PySpark-d414eddb-389e-437a-a1f7-f7bb7f467b8b/pyspark_runner.py", line 134, in <module>
    eval(compiled_code)
  File "<string>", line 4, in <module>
  File "//anaconda/lib/python2.7/site-packages/sklearn/ensemble/forest.py", line 649, in predict
    X = self._validate_X_predict(X)
  File "//anaconda/lib/python2.7/site-packages/sklearn/ensemble/forest.py", line 319, in _validate_X_predict
    return self.estimators_[0]._validate_X_predict(X, check_input=True)
  File "//anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py", line 365, in _validate_X_predict
    X = check_array(X, dtype=DTYPE, accept_sparse="csr")
  File "//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py", line 398, in check_array
    _assert_all_finite(array)
  File "//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py", l